In [ ]:
!pip install openai==0.28.1 python-dotenv pandas

In [ ]:
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
import os
#os.getenv("OPENAI_API_KEY")

In [ ]:
#modelo_gpt = "gpt-3.5-turbo"
# modelo_gpt = "gpt-3.5-turbo-1106"
# modelo_gpt = "gpt-4"
modelo_gpt = "gpt-4-1106-preview"

In [ ]:
# Testando a comunicação com a OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
resultado = openai.ChatCompletion.create(
    model = modelo_gpt,
    messages = [{"role": "user", "content": "Qual a importância de se fazer uma Pós-graduação em Análise de Dados? Responda de forma bem sucinta."}],
)

display(resultado.choices[0].message.content)

In [ ]:
# MetaPrompt
system_prompt = """Você é um assistente de análise de dados. Dado o relato de um fato feito por uma vítima que conversou com a ouvidoria de um banco, \
  você fará o TAG / classificação do relato exclusivamente em uma das categorias listadas abaixo. Responda EXCLUSIVAMENTE com o TAG, sem complementação \
  sobre a categoria listada. Se o fato não se enquadrar em nenhuma das categorias listadas, responda com NAO_CLASSIFICADO.


Lista taxativa de Categorias:
APLICATIVO HACKEADO,
APLICATIVO OU PAGINA FAKE,
COMPROVANTE FALSO,
CPF CLONADO,
FALSA INDENIZACAO,
FALSA DOACAO,
FALSA NAMORADA,
FALSA NOTA FISCAL,
FALSA VENDA DE PRODUTO OU SERVIÇO,
FALSO ANTENDENTE BANCARIO,
FALSO ANUNCIO,
FALSO ATENDENTE,
FALSO COMPROVANTE,
FALSO CONTRABANDO,
FALSA OFERTA DE EMPREGO,
FALSO EMPRESTIMO,
FALSO EXAME MEDICO,
FALSO MEDICO,
FALSO PREMIO,
FALSO SEGURO,
FALSO SEQUESTRO,
INVESTIMENTO MILAGROSO,
FALSO PARENTE OU AMIGO EM NECESSIDADE,
FALSO SEQUESTRO,
FALSA DÍVIDA,
EXTORSAO,
NAO_CLASSIFICADO
"""

In [ ]:
# Template de prompt para as requisições do usuário
def meu_prompt_template(relato_vitima):
  return f"""
Relato: {relato_vitima}


Categoria Selecionada da Lista:
"""

In [ ]:
contador = 0
# Criar uma função de classificação
def classifica_tipologia(registro_vitima):
    global system_prompt
    global contador

    if registro_vitima is None or registro_vitima.strip() == "":
        return "NAO CLASSIFICADO"
    prompt = meu_prompt_template(registro_vitima)

    contador += 1
    print(f"{contador}:", end='')
    print(prompt)

    resultado = openai.ChatCompletion.create(
        model = modelo_gpt,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens = 1000
    )
    classificacao = resultado.choices[0].message.content;

    print(classificacao)
    print("--------------")
    print()

    return classificacao

print(f"Será usado o modelo {modelo_gpt}")

In [ ]:
#  leitura dos dados
import pandas as pd
pd_tipologias = pd.read_excel("Demonstração Categorização IA.xlsx", sheet_name="OCORRENCIAS")

In [ ]:
contador = 0;
#  função (transformação) sobre uma coluna, gerando uma nova coluna
pd_tipologias["TIPOLOGIA"] = pd_tipologias["REGISTRO DA VÍTIMA"].apply(lambda x: classifica_tipologia(x))

In [ ]:
# Salvar os resultados para uma nova planilha

pd_tipologias.to_excel(f"Categorização IA - resultado_{modelo_gpt}.xlsx", index=False)